# Napari Visualization Pipeline

In [172]:
import napari
import numpy as np
from skimage import morphology, restoration, filters, measure
from tifffile import imread

#img = imread('cell-data/parent_control_z3_ch00.tif') --> This was an initial testing image

# Current testing images
nuclei_img = imread('cell-data/test-image-set/22KO_mCHC17_GFP_3_z3_ch00.tif')
transfection_img = imread('cell-data/test-image-set/22KO_mCHC17_GFP_3_z3_ch01.tif')
p115_img = imread('cell-data/test-image-set/22KO_mCHC17_GFP_3_z3_ch02.tif')

viewer = napari.Viewer()
viewer.add_image(nuclei_img, name='nuclei')
viewer.theme = 'light'

### Erosion

In [173]:
from skimage.morphology import erosion, disk

eroded = erosion(img, disk(1)) # Change to 2 or 3 for more intense erosion

viewer.add_image(eroded, name='eroded')

<Image layer 'eroded' at 0x374115310>

### Estimate noise and denoise via non-local means

In [174]:
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.util import img_as_float

img_float = img_as_float(eroded)

sigma_est = np.mean(estimate_sigma(img_float, channel_axis=None)) # Estimated noise standard deviation. Lower SD means lower estimated noise.

denoised = denoise_nl_means(
    img_float,
    h=0.8 * sigma_est, # h is filter strength. This line sets filter strength to 80% of estimated noise level, but tweak this number if necessary.
    patch_size=5, # Larger value = captures more context; better denoising but loses fine detail (probably doesn't matter for this use).
    patch_distance=6, # Larger value = searches wider area, so better denoising but slower processing.
    fast_mode=True # Make False for more accurate denoising (but it'll be a bit slower).
)

viewer.add_image(denoised, name='denoised')

<Image layer 'denoised' at 0x3719421e0>

### Create nucleus mask

In [175]:
threshold = 0.1 # Play around with the threshold to find the optimal value.
thresholded = denoised > threshold # For each pixel --> if it's brighter than the threshold value, count it as foreground (part of a nucleus).
# The line above creates a boolean array.

viewer.add_labels(thresholded.astype(int), name='thresholded') # This turns the boolean array into binaries.

<Labels layer 'thresholded' at 0x31f0fb4a0>

### Fill any gaps in the mask regions

In [176]:
from scipy.ndimage import binary_fill_holes
import numpy as np

# Assuming 'mask' is a binary numpy array where nuclei are True (1) and background is False (0)
filled_mask = binary_fill_holes(thresholded)

# Convert back to int if needed
filled_mask = filled_mask.astype(np.uint8)

viewer.add_labels(filled_mask, name='hole-filled-mask')

<Labels layer 'hole-filled-mask' at 0x36ed87ce0>

### Separate mask into distinct regions with integer labels

In [177]:
from skimage.measure import label

labeled_mask = label(filled_mask) # Label connected regions of the mask

viewer.add_labels(labeled_mask, name='nuclei_mask')

<Labels layer 'nuclei_mask' at 0x371c8ee10>

### Clean mask by removing nucleus regions that are cut off by the image and by removing isolated pixels that remain after erosion and denoising

In [178]:
from skimage.measure import regionprops
import numpy as np

image_height, image_width = labeled_mask.shape
cleaned_mask = np.zeros_like(labeled_mask)

for region in regionprops(labeled_mask):
    min_row, min_column, max_row, max_column = region.bbox

    # This checks for labeled regions at the edges of the image
    touches_edge = (min_row == 0 or min_column == 0 or max_row == image_height or max_column == image_width)
    
    if not touches_edge and region.area >= 500:
        cleaned_mask[labeled_mask == region.label] = region.label 
        # This line sets all pixels in cleaned_mask at positions where the boolean mask is True to the integer value region.label.
        # Each specific integer value is a specific label/color.
        
viewer.add_labels(cleaned_mask, name='cleaned_mask')

<Labels layer 'cleaned_mask' at 0x36be4fe60>

### Determine the image's microns per pixel ratio and create the first donut mask

In [179]:
from skimage.morphology import dilation, disk
from skimage.segmentation import find_boundaries
import tifffile

with tifffile.TiffFile('cell-data/parent_control_z3_ch00.tif') as tif:
    page = tif.pages[0]
    tags = page.tags

    x_res = tags['XResolution'].value  # (num, denom)
    y_res = tags['YResolution'].value
    res_unit = tags['ResolutionUnit'].value  # 3 = centimeters

    # Convert to pixels per micrometer
    if res_unit == 3:  # Centimeters
        x_ppcm = x_res[0] / x_res[1]
        y_ppcm = y_res[0] / y_res[1]
        x_ppum = x_ppcm / 10000  # cm → µm
        y_ppum = y_ppcm / 10000
        print('Compare these values to the metadata in Fiji ImageJ:')
        print(f'X pixels/µm: {x_ppum:.4f}')
        print(f'Y pixels/µm: {y_ppum:.4f}')
    else:
        print('Unsupported resolution unit:', res_unit)

microns_per_pixel = (1 / x_ppum + 1 / y_ppum) / 2
desired_donut_width_um = 2
n_pixels = int(desired_donut_width_um / microns_per_pixel)

outer_mask = dilation(cleaned_mask, disk(n_pixels))

donut_mask = outer_mask - cleaned_mask
donut_mask[donut_mask < 0] = 0  # Just in case

donut_mask = donut_mask.astype(np.uint8)

viewer.add_labels(donut_mask, name='donut')

Compare these values to the metadata in Fiji ImageJ:
X pixels/µm: 5.5081
Y pixels/µm: 5.5081


<Labels layer 'donut' at 0x36e095820>

### Create the outer donut mask

In [180]:
desired_outer_width_um = 3
n_pixels = int(desired_outer_width_um / microns_per_pixel)

outer_donut_mask = np.zeros_like(cleaned_mask, dtype=np.uint16)

regions = regionprops(cleaned_mask)

for region in regions:
    nucleus_mask = (cleaned_mask == region.label)
    first_donut_mask = (donut_mask == region.label)
    excluded_area = nucleus_mask | first_donut_mask
    outer_mask = dilation(excluded_area, disk(n_pixels))
    outer_donut = outer_mask & (~excluded_area)
    outer_donut_mask[outer_donut] = region.label

viewer.add_labels(outer_donut_mask, name='outer_donut')

<Labels layer 'outer_donut' at 0x373f179e0>

### Delete all overlapping regions --> REVIEW THIS CODE ENTIRELY TO FULLY UNDERSTAND IT

In [187]:
from skimage.morphology import square

# Step 1: Get all unique labels
labels = np.unique(cleaned_mask)
labels = labels[labels != 0]

# Step 2: Build combined masks per region
regions = []
for label_id in labels:
    combined_mask = (
        (cleaned_mask == label_id) |
        (donut_mask == label_id) |
        (outer_donut_mask == label_id)
    )
    regions.append({
        'label': label_id,
        'mask': combined_mask
    })

# Step 3: Find touching regions using dilation
labels_to_remove = set()

for i, region_a in enumerate(regions):
    dilated_a = dilation(region_a['mask'], square(3))  # 8-connected dilation

    for j, region_b in enumerate(regions):
        if i >= j:
            continue  # avoid duplicate checks

        if np.any(dilated_a & region_b['mask']):
            labels_to_remove.add(region_a['label'])
            labels_to_remove.add(region_b['label'])

# Step 4: Remove touching labels from all masks
def remove_labels(mask, labels_to_remove):
    output = np.copy(mask)
    output[np.isin(mask, list(labels_to_remove))] = 0
    return output.astype(np.int32)

cleaned_filtered = remove_labels(cleaned_mask, labels_to_remove)
donut_filtered = remove_labels(donut_mask, labels_to_remove)
outer_donut_filtered = remove_labels(outer_donut_mask, labels_to_remove)

# Step 5: Combine into final mask
no_overlap_mask = cleaned_filtered + donut_filtered + outer_donut_filtered
viewer.add_labels(no_overlap_mask, name="no_overlap_mask")

/var/folders/zc/r5_5f1x56sq9gtmyssh_0g300000gn/T/ipykernel_36213/207430229.py:24: FutureWarning: `square` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  dilated_a = dilation(region_a['mask'], square(3))  # 8-connected dilation


<Labels layer 'no_overlap_mask' at 0x37144eab0>

### Note:
Make the alternative overlapping approach (where instead of removing any and all overlapping regions, just delete regions that overlap with inner donuts).

### Delete regions with cut-off donuts --> edit this and the cell above such that there are still 3 different layers (nucleus, inner donut, outer donut) because the next step involves looking ONLY at the inner donut.

In [190]:
image_height, image_width = no_overlap_mask.shape
no_edges_no_overlaps = np.zeros_like(no_overlap_mask)

for region in regionprops(no_overlap_mask):
    min_row, min_column, max_row, max_column = region.bbox

    touches_edge = (min_row == 0 or min_column == 0 or max_row == image_height or max_column == image_width)
    
    if not touches_edge and region.area >= 500:
        no_edges_no_overlaps[no_overlap_mask == region.label] = region.label 
        
viewer.add_labels(no_edges_no_overlaps, name='no_edges_no_overlaps')

<Labels layer 'no_edges_no_overlaps [1]' at 0x373ffa1e0>

### Remove cells with low transfection

In [192]:
viewer.add_image(
    transfection_img,
    name="transfection",
    colormap="green",         # or "magma", "viridis", "cividis", etc.
    contrast_limits=[0, 255], # or adjust depending on intensity range
    blending="additive",      # good for overlays
    opacity=0.7               # adjust as needed
)

<Image layer 'transfection' at 0x372f2eed0>